# Explore Creating Network from OSM

Note this is an exploratory notebook for if you want to create the networks step-by-step and review and customize parts along the way.

If you'd like to accept some gross defaults, then you can use `bin/build_basic_osm_roadnet.py` which puts all of this together.

In [8]:
import osmnx as ox

In [42]:
g = ox.graph_from_place('Piedmont, California, USA', network_type='drive')

In [117]:
nodes, edges = ox.graph_to_gdfs(g)

In [ ]:
mymap = edges.explore(tiles = 'cartodb positron', color='blue', column='lanes', cmap='tab20', legend=True)
nodes.explore(m=mymap, color='grey', alpha=0.5)

#### Checking uniqueness

When checking for uniqueness in uv, it looks like all of these are loops where it would be fine to delete the longer one for the purposes of routing....so that's what we will do.

In [ ]:
edges.reset_index(inplace=True)
edges.loc[edges.duplicated(subset=['u','v'], keep=False)].sort_values(by=['u','v'])

In [92]:
_links_df = edges.reset_index().sort_values('geometry').drop_duplicates(subset=['u','v'], keep='first')

### Rename variables

In [95]:
rename_cols = {
    'u': 'A',
    'v': 'B',
    'name': 'name',
    'geometry': 'geometry',
    'lanes': 'lanes',
    'highway': 'roadway',
    'osmid': 'osm_link_id',
}
links_df  = _links_df.loc[:, list(rename_cols.keys())].rename(columns=rename_cols)

#### Create Variables

- Fill in lanes based on a lookup table
- Create model_link_id from a hash of A and B

In [101]:
def get_min_lane_value(lane):
    if isinstance(lane, list):
        return min(lane)
    return lane

links_df['lanes'] = links_df['lanes'].apply(get_min_lane_value)

In [ ]:
links_df['lanes'].value_counts()

In [ ]:
pd.crosstab(links_df['roadway'], links_df['lanes'])

In [104]:
lanes_lookup = {
    'residential': 1,
    'tertiary': 1,
    'secondary': 2,
    'primary': 2,
    'trunk': 2,
    'motorway': 3,
    'motorway_link': 1,
    'primary_link': 1,
    'secondary_link': 1,
    'tertiary_link': 1,
    'trunk_link': 1,
    'service': 1,
    'unclassified': 1,
    'footpath': 0,
    'cycleway': 0,
    'busway': 1,
    'steps': 0,
}
links_df['lanes'] = links_df['lanes'].fillna(links_df['roadway'].map(lanes_lookup))


In [109]:
import hashlib

def generate_unique_id(row):
    unique_string = f"{row['A']}_{row['B']}"
    return int(hashlib.sha256(unique_string.encode()).hexdigest(), 16) % 10**8

links_df['model_link_id'] = links_df.apply(generate_unique_id, axis=1)
assert links_df['model_link_id'].is_unique, "model_link_id values are not unique"

### Add access variables
Based on roadway type (although the network I pulled from OSM is drive access already)

In [110]:
links_df["rail_only"] = links_df.roadway.str.contains("rail")
links_df["walk_access"] = links_df.roadway.isin(['footway','path','steps'])
links_df["bike_access"] = links_df.roadway.isin(['path','cycleway','service','tertiary','residential'])
links_df["drive_access"] = links_df.roadway.isin(['footway','path','steps','cycleway',None])
links_df["truck_access"] = links_df["drive_access"]


In [ ]:
links_df.explore(tiles = 'CartoDB dark_matter', column='lanes', cmap='viridis', legend=True)

This clearly doesn't look right for lanes. I think we should apply the lookup to ALL links, not just NA.

In [ ]:
links_df['lanes'] = links_df['roadway'].map(lanes_lookup)
links_df.explore(tiles = 'CartoDB dark_matter', column='lanes', cmap='viridis', legend=True)

In [137]:
links_df.A = links_df.A.astype(int)
links_df.B = links_df.B.astype(int)

## Create Nodes

In [ ]:
nodes = nodes.reset_index()
nodes['model_node_id'] = nodes.osmid.astype(int)
nodes = nodes.rename(columns={'osmid': 'osm_node_id'})
nodes.model_node_id.is_unique

In [151]:
column_rename_keep = {
    'model_node_id': 'model_node_id',
    'osm_node_id': 'osm_node_id',
    'x': 'X',
    'y': 'Y',
    'geometry': 'geometry',
}
nodes_df = nodes.loc[:, list(column_rename_keep.keys())].rename(columns=column_rename_keep)
nodes_df.head()

## Import to Network Wrangler

In [153]:
from network_wrangler.roadway.links.create import data_to_links_df
from network_wrangler.roadway.network import RoadwayNetwork

wr_links_df = data_to_links_df(links_df, nodes_df=nodes_df)
road_net = RoadwayNetwork(nodes_df=nodes_df, links_df=wr_links_df)
